In [1]:
import pandas as pd
import numpy as np
import sklearn
from keras.models import Sequential
from keras.layers import Dense, Dropout, Masking, Embedding
from matplotlib import pyplot as plt
from ipywidgets import interact
import pickle
plt.rcParams['figure.dpi'] = 200

In [2]:
file_headers = ["/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext1",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext2",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext3",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext4",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext1",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext2",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext3",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext4",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/s_curve1",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/s_curve2"]

CSV_SFX = ".csv"
MARKERS_SFX = "_markers"
POLY_SFX = "_poly"
M10Y = "M10Y"

In [3]:
# distance goes from 22-38. 16 cm of extension at least which is pretty good.

In [4]:
poly_df_list = []
marker_df_list = []
data_df_list = []

for header in file_headers:
    data_file = header + CSV_SFX
    marker_file = header + MARKERS_SFX + CSV_SFX
    poly_file = header + POLY_SFX + CSV_SFX
    
    data_df = pd.read_csv(data_file)
    marker_df = pd.read_csv(marker_file)
    poly_df = pd.read_csv(poly_file)
    
    zero_indices = list(marker_df[marker_df[M10Y] == 0].index)
    
    data_df = data_df.drop(data_df.index[zero_indices]).reset_index()
    marker_df = marker_df.drop(marker_df.index[zero_indices])
    poly_df = poly_df.drop(poly_df.index[zero_indices])
    
    data_df_list.append(data_df)
    marker_df_list.append(marker_df)
    poly_df_list.append(poly_df)

total_data_df = pd.concat(data_df_list).reset_index()
total_marker_df = pd.concat(marker_df_list).reset_index()
total_poly_df = pd.concat(poly_df_list).reset_index()


training_data_list = [total_data_df, total_marker_df, total_poly_df]
all_df = pd.concat(training_data_list, axis=1)

x_labels = ["M1-PL", "M1-PR", "M2-PL", "M2-PR"]
# NOTE: Uncomment if using polynomial
# y_labels = ["a0", "a1", "a2", "a3", "a4", "a5", "d"]

# NOTE: Uncomment if using markers
y_labels = ["M1X", "M1Y", "M2X", "M2Y", "M3X", "M3Y",
            "M4X", "M4Y", "M5X", "M5Y", "M6X", "M6Y",
            "M7X", "M7Y", "M8X", "M8Y", "M9X", "M9Y",
            "M10X", "M10Y"]

def normalize_columns(df, col_list):
    min_vals = []
    max_vals = []
    for c in col_list:
        min_val = df[c].min()
        max_val = df[c].max()
        min_vals.append(min_val)
        max_vals.append(max_val)
        df[c] = (df[c] - min_val) / (max_val - min_val)
    return min_vals, max_vals
        
x_mins, x_maxes = normalize_columns(all_df, x_labels)
y_mins, y_maxes = normalize_columns(all_df, y_labels)

In [5]:
# Prepare data for DNN
# shuffle the rows
#train_df = all_df.sample(frac=0.8, random_state=25)
#test_df = all_df.drop(train_df.index)

x_dim = len(x_labels)
y_dim = len(y_labels)

x_data = all_df[x_labels]
y_data = all_df[y_labels]

x_data_shuffled, y_data_shuffled = sklearn.utils.shuffle(x_data, y_data)

split = int(0.8 * len(x_data))

x_train = x_data_shuffled[:split]
y_train = y_data_shuffled[:split]

x_test = x_data_shuffled[split:]
y_test = y_data_shuffled[split:]

In [7]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(4895, 4)
(1224, 4)
(4895, 20)
(1224, 20)


# DNN

In [8]:
model = Sequential()
model.add(Dense(64 , input_dim=x_dim, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, input_dim=16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_dim, activation='relu'))
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                320       
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 20)                3

In [9]:
history = model.fit(x_train, y_train, epochs=100, batch_size=8)

Epoch 1/100
612/612 [==============================] - 1s 411us/step - loss: 0.0887 - accuracy: 0.1416
Epoch 2/100
612/612 [==============================] - 0s 407us/step - loss: 0.0221 - accuracy: 0.2766
Epoch 3/100
612/612 [==============================] - 0s 407us/step - loss: 0.0168 - accuracy: 0.3190
Epoch 4/100
612/612 [==============================] - 0s 410us/step - loss: 0.0156 - accuracy: 0.3862
Epoch 5/100
612/612 [==============================] - 0s 402us/step - loss: 0.0139 - accuracy: 0.4251
Epoch 6/100
612/612 [==============================] - 0s 403us/step - loss: 0.0135 - accuracy: 0.4457
Epoch 7/100
612/612 [==============================] - 0s 419us/step - loss: 0.0130 - accuracy: 0.4711
Epoch 8/100
612/612 [==============================] - 0s 414us/step - loss: 0.0127 - accuracy: 0.4882
Epoch 9/100
612/612 [==============================] - 0s 409us/step - loss: 0.0122 - accuracy: 0.4796
Epoch 10/100
612/612 [==============================] - 0s 414us/step - l

612/612 [==============================] - 0s 406us/step - loss: 0.0070 - accuracy: 0.5636
Epoch 80/100
612/612 [==============================] - 0s 405us/step - loss: 0.0075 - accuracy: 0.5674
Epoch 81/100
612/612 [==============================] - 0s 404us/step - loss: 0.0072 - accuracy: 0.5533
Epoch 82/100
612/612 [==============================] - 0s 403us/step - loss: 0.0069 - accuracy: 0.5399
Epoch 83/100
612/612 [==============================] - 0s 403us/step - loss: 0.0071 - accuracy: 0.5588
Epoch 84/100
612/612 [==============================] - 0s 409us/step - loss: 0.0075 - accuracy: 0.5393
Epoch 85/100
612/612 [==============================] - 0s 406us/step - loss: 0.0072 - accuracy: 0.5620
Epoch 86/100
612/612 [==============================] - 0s 417us/step - loss: 0.0065 - accuracy: 0.5730
Epoch 87/100
612/612 [==============================] - 0s 409us/step - loss: 0.0071 - accuracy: 0.5524
Epoch 88/100
612/612 [==============================] - 0s 412us/step - loss:

In [10]:
loss, accuracy = model.evaluate(x_test, y_test)
print('Loss: %.8f, Accuracy: %.2f' % (loss, (accuracy*100)))

39/39 [==============================] - 0s 345us/step - loss: 0.0032 - accuracy: 0.6250
Loss: 0.00322711, Accuracy: 62.50


In [35]:
pickle.dump(history.history["loss"], open("objects/dnn_loss.p", "wb"))

In [11]:
def rescale_output(y):
    ret = []
    for i in range(len(y)):
        max_val = y_maxes[i]
        min_val = y_mins[i]
        val = y[i]
        rescaled_val = (val * (max_val - min_val)) + min_val
        ret.append(rescaled_val)
    return np.array(ret)

In [37]:
num_tests = len(x_test)
def evaluate_poly(idx=(0,num_tests-1,1)):
    index = x_test.index[idx]
    x_select = np.array(x_test.loc[index, :]).reshape(1, -1)
    y_pred = model.predict(x_select)
    y_pred = rescale_output(y_pred[0])
    a_pred = np.flip(y_pred[:-1])
    d_pred = y_pred[-1]
    
    y_select = np.array(y_test.loc[index, :]).reshape(1, -1)
    y_select = rescale_output(y_select[0])
    a_select = np.flip(y_select[:-1])
    d_select = y_select[-1]
    
    print(d_pred, d_select)
    
    poly_pred = np.poly1d(a_pred)
    poly_select = np.poly1d(a_select)
    
    yp = np.linspace(0, d_pred)
    xp = poly_pred(yp)
    
    ys = np.linspace(0, d_select)
    xs = poly_select(ys)
    
    plt.plot(xs, ys, label="actual")
    plt.plot(xp, yp, label="predicted")
    plt.xlim([-20,20])
    plt.ylim([-1,39])
    plt.xlabel("x (cm)")
    plt.ylabel("y (cm)")
    plt.legend()
    plt.title("Predicted Shape vs. Actual Shape")
    plt.show()
    

interact(evaluate_poly)

Widget Javascript not detected.  It may not be installed or enabled properly.


<function __main__.evaluate_poly(idx=(0, 1223, 1))>

In [15]:
num_tests = len(x_test)
def evaluate_markers(idx=(0,num_tests-1,1)):
    index = x_test.index[idx]
    x_select = np.array(x_test.loc[index, :]).reshape(1, -1)
    y_pred = model.predict(x_select)
    y_pred = rescale_output(y_pred[0])
    
    y_select = np.array(y_test.loc[index, :]).reshape(1, -1)
    y_select = rescale_output(y_select[0])
    
    num_markers = int(len(y_pred) / 2)
    
    x_coord_select = [0]
    y_coord_select = [0]
    x_coord_pred = [0]
    y_coord_pred = [0]
    
    for i in range(num_markers):
        x_idx = i * 2
        y_idx = (i * 2) + 1
        x_coord_select.append(y_select[x_idx])
        y_coord_select.append(y_select[y_idx])
        x_coord_pred.append(y_pred[x_idx])
        y_coord_pred.append(y_pred[y_idx])
    
    plt.plot(x_coord_select, y_coord_select, "-o", label="actual")
    plt.plot(x_coord_pred, y_coord_pred, "-o", label="predicted")
    plt.xlim([-20,20])
    plt.ylim([-1,39])
    plt.xlabel("x (cm)")
    plt.ylabel("y (cm)")
    plt.legend()
    plt.title("Predicted Markers vs. Actual Markers")
    plt.show()
        
        
interact(evaluate_markers)

Widget Javascript not detected.  It may not be installed or enabled properly.


<function __main__.evaluate_poly(idx=(0, 1223, 1))>

In [149]:
model.save("./models/first_model")

INFO:tensorflow:Assets written to: ./models/first_model/assets


In [ ]:
# First model tried
model = Sequential()
model.add(Dense(12, input_dim=x_dim, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(y_dim, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])